# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
- アイテムあたりの「週Or日」平均販売個数 累積　2，3，6，12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり 170Item 販売個数累積　-> MinMaxScaler　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり 170Item 販売個数累積　-> MinMaxScaler 'date_cat_avg_item_cnt_lag_1',
- アイテムカテゴ/ショップあたり 170Item 販売個数累積　-> MinMaxScaler 'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり 170Item 販売個数累積　-> MinMaxScaler 'date_cat_avg_item_cnt_lag_1',
- カテゴリ/ショップあたり 170Item  販売個数累積　MinMaxScaler
- ジャンル　あたり 170Item 販売個数累積　-> MinMaxScaler
- ジャンル/ショップ　170Item 販売個数累積　　MinMaxScaler

- ASP　月当たり？
- 価格下落率?
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

sales_history = pd.read_csv('sales_history.csv')
sales_history.columns = ['Date', 'StoreId', 'ItemId', 'ItemPrice', 'SoldNum']
sales_history['Date'] = pd.to_datetime(sales_history['Date'])

test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
#item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')
'''170品目のリスト'''
test_item = test['ItemId'].unique()
test_item = pd.DataFrame({'ItemId':test_item})
#test_item
'''カテゴリ名と連結'''
ttest = pd.merge(test,item_categories, on='ItemId')
ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('予測データの品目数: ',ttest['ItemId'].nunique(),'\n',
      '訓練データの品目数: ',sales_history['ItemId'].nunique(),'\n',
      '予測データのカテゴリ数: ',item_categories['ItemCategoryId'].nunique(),'\n',
      '訓練データのカテゴリ数: ',ttest['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',sales_history['SoldNum'].sum(), '\n',
     '')

予測データの品目数:  170 
 訓練データの品目数:  9040 
 予測データのカテゴリ数:  26 
 訓練データのカテゴリ数:  26 
 売上の総数:  1262869.0 
 


# 休日フラグ  sales_total.csvとして保存
https://upura.hatenablog.com/entry/2018/12/21/070000

In [3]:
sales_total = pd.read_csv('./csv/sales_total.csv')
'''
from datetime import date,timedelta
import jpholiday
%time
sales_total['is_holiday'] = sales_total['Date'].map(jpholiday.is_holiday).astype(int)'''

"\nfrom datetime import date,timedelta\nimport jpholiday\n%time\nsales_total['is_holiday'] = sales_total['Date'].map(jpholiday.is_holiday).astype(int)"

In [4]:
sales_total

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0
1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0
1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0
1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0


In [5]:
#sales_total.to_csv('sales_total.csv',index=False)

# 年月にグループ化

In [6]:
sales_total['Date'] = pd.to_datetime(sales_total['Date'])
sales_total['YYYYMM'] = sales_total['Date'].dt.strftime('%Y%m')

In [7]:
#sales_total.info()

# 単価→ASP , 売上個数集計

In [8]:
monthly_sales = sales_total.groupby([
    'YYYYMM','StoreId','ItemId','ItemCategoryId','Category',
    'Type']).agg({'ItemPrice':'mean','SoldNum':'sum','is_holiday':'sum'}).reset_index()

In [9]:
monthly_sales

,YYYYMM,StoreId,ItemId,ItemCategoryId,Category,Type,ItemPrice,SoldNum,is_holiday
0,201801,0,1000001,100,5,1,420.0,6.0,0
1,201801,0,1000002,100,5,1,250.0,2.0,0
2,201801,0,1000004,100,5,1,120.0,1.0,0
3,201801,0,1000005,100,5,1,220.0,8.0,0
4,201801,0,1000006,100,5,1,190.0,2.0,0
...,...,...,...,...,...,...,...,...,...
495129,201910,17,3400005,340,4,9,1190.0,1.0,0
495130,201910,17,3400006,340,4,9,2700.0,1.0,0
495131,201910,17,3400012,340,4,9,680.0,1.0,0
495132,201910,17,3400016,340,4,9,250.0,1.0,0


# 月別販売個数170アイテムデータ

In [10]:
monthly_soldnum = pd.read_csv('./csv/monthly_soldnum.csv')
monthly_soldnum = monthly_soldnum.drop(columns= monthly_soldnum.columns[0])
monthly_soldnum = monthly_soldnum.drop(columns= 'index')

In [11]:
monthly_soldnum

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,2.0,1.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,5.0,2.0,3.0,2.0,5.0,4.0,5.0,6.0,3.0,2.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,4.0,8.0,6.0,3.0,5.0,6.0,9.0,14.0,8.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,0.0,2.0,1.0,0.0,2.0,1.0,1.0,1.0,3.0,2.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 月次売上個数　移動平均

In [12]:
T12 = monthly_soldnum.rolling(12,axis=1).sum()
T6 = monthly_soldnum.rolling(6,axis=1).sum()
T3 = monthly_soldnum.rolling(3,axis=1).sum()
T2 = monthly_soldnum.rolling(2,axis=1).sum()

monthly_soldnum['T2'] = T2['201910']
monthly_soldnum['T3'] = T3['201910']
monthly_soldnum['T6'] = T6['201910']
monthly_soldnum['T12'] = T12['201910']
monthly_soldnum_T = monthly_soldnum

In [13]:
monthly_soldnum_T

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201905,201906,201907,201908,201909,201910,T2,T3,T6,T12
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,3.0,8.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,2.0,0.0,3.0,0.0,2.0,2.0,5.0,7.0,12.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,5.0,4.0,5.0,6.0,3.0,2.0,5.0,11.0,25.0,53.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,5.0,6.0,9.0,14.0,8.0,12.0,20.0,34.0,54.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,2.0,1.0,1.0,1.0,3.0,2.0,5.0,6.0,10.0,17.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
#monthly_soldnum_T.to_csv('./csv/monthly_soldnum_T.csv')

In [15]:
monthly_sales

,YYYYMM,StoreId,ItemId,ItemCategoryId,Category,Type,ItemPrice,SoldNum,is_holiday
0,201801,0,1000001,100,5,1,420.0,6.0,0
1,201801,0,1000002,100,5,1,250.0,2.0,0
2,201801,0,1000004,100,5,1,120.0,1.0,0
3,201801,0,1000005,100,5,1,220.0,8.0,0
4,201801,0,1000006,100,5,1,190.0,2.0,0
...,...,...,...,...,...,...,...,...,...
495129,201910,17,3400005,340,4,9,1190.0,1.0,0
495130,201910,17,3400006,340,4,9,2700.0,1.0,0
495131,201910,17,3400012,340,4,9,680.0,1.0,0
495132,201910,17,3400016,340,4,9,250.0,1.0,0


# ショップごとの販売個数

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
store_sales = monthly_sales.groupby(['StoreId']).agg({'SoldNum':np.sum}).reset_index()
store_sales =store_sales.rename(columns={'SoldNum':'StoreSalesSum'})
store_sales

,StoreId,StoreSalesSum
0,0,107953.0
1,1,46347.0
2,2,56431.0
3,3,87529.0
4,4,64902.0
5,5,54031.0
6,6,102087.0
7,7,124894.0
8,8,41841.0
9,9,67352.0


In [18]:
#store_sales.to_csv('./csv/store_sales.csv')

# ItemCategory あたりの販売個数　→MinMaxScaler

In [19]:
ItemCategoryId_sales = monthly_sales.groupby(['ItemCategoryId']).agg({'SoldNum':np.sum}).reset_index()
ItemCategoryId_sales = ItemCategoryId_sales.rename(columns={'SoldNum':'ItemCategoryIdSoldSum'})
ItemCategoryId_sales 

,ItemCategoryId,ItemCategoryIdSoldSum
0,100,293135.0
1,110,204775.0
2,120,165089.0
3,130,127512.0
4,140,94123.0
5,150,65304.0
6,160,31815.0
7,170,23059.0
8,180,21968.0
9,190,24741.0


In [20]:
#ItemCategoryId_sales.to_csv('./csv/ItemCategoryId_sales.csv')

# Category あたりの販売個数　→MinMaxScaler

In [21]:
Category_sales = monthly_sales.groupby(['Category']).agg({'SoldNum':np.sum}).reset_index()
Category_sales = Category_sales.rename(columns={'SoldNum':'CategorySalesSum'})
Category_sales

,Category,CategorySalesSum
0,0,24741.0
1,1,153816.0
2,2,204775.0
3,3,223326.0
4,4,14502.0
5,5,427033.0
6,6,27750.0
7,7,186926.0


In [22]:
#Category_sales.to_csv('./csv/Category_sales.csv')

# Type あたりの販売個数　→MinMaxScaler

In [23]:
Type_sales = monthly_sales.groupby(['Type']).agg({'SoldNum':np.sum}).reset_index()
Type_sales = Type_sales.rename(columns={'SoldNum':'TypeSalesSum'})
Type_sales

,Type,TypeSalesSum
0,0,16874.0
1,1,293135.0
2,2,24741.0
3,3,165089.0
4,4,21351.0
5,5,14139.0
6,6,2934.0
7,7,23059.0
8,8,17137.0
9,9,26510.0


In [24]:
#Type_sales.to_csv('./csv/Type_sales.csv')

 # アイテムカテゴリ/ショップあたり  販売個数累積　

In [25]:
StoreId_ItemCategoryId_sales = monthly_sales.groupby(
    ['StoreId','ItemCategoryId']).agg({'SoldNum':np.sum}).reset_index()
StoreId_ItemCategoryId_sales = StoreId_ItemCategoryId_sales.rename(
    columns={'SoldNum':'Store_ItemCat_SalesSum'})
StoreId_ItemCategoryId_sales

,StoreId,ItemCategoryId,Store_ItemCat_SalesSum
0,0,100,27840.0
1,0,110,12245.0
2,0,120,13726.0
3,0,130,11926.0
4,0,140,8814.0
...,...,...,...
463,17,310,498.0
464,17,320,516.0
465,17,330,405.0
466,17,340,220.0


In [26]:
#StoreId_ItemCategoryId_sales.to_csv('./csv/StoreId_ItemCategoryId_sales.csv')

 # カテゴリ/ショップあたり  販売個数累積　

In [27]:
StoreId_Category_sales = monthly_sales.groupby(
    ['StoreId','Category']).agg({'SoldNum':np.sum}).reset_index()
StoreId_Category_sales= StoreId_Category_sales.rename(columns={'SoldNum':'Store_Cat_SalesSum'})
StoreId_Category_sales

,StoreId,Category,Store_Cat_SalesSum
0,0,0,1239.0
1,0,1,14955.0
2,0,2,12245.0
3,0,3,19158.0
4,0,4,1108.0
...,...,...,...
139,17,3,12280.0
140,17,4,736.0
141,17,5,15230.0
142,17,6,415.0


In [28]:
#StoreId_Category_sales.to_csv('./csv/StoreId_Category_sales.csv')

 # Type /ショップあたり  販売個数累積

In [29]:
StoreId_Type_sales = monthly_sales.groupby(
    ['StoreId','Type']).agg({'SoldNum':np.sum}).reset_index()
StoreId_Type_sales = StoreId_Type_sales.rename(columns={'SoldNum':'StoreTypeSalesSum'})
StoreId_Type_sales

,StoreId,Type,StoreTypeSalesSum
0,0,0,1847.0
1,0,1,27840.0
2,0,2,1239.0
3,0,3,13726.0
4,0,4,1462.0
...,...,...,...
427,17,19,1235.0
428,17,20,149.0
429,17,21,369.0
430,17,22,299.0


In [30]:
StoreId_Type_sales.to_csv('./csv/StoreId_Type_sales.csv')

# 月次売上・移動平均データに特徴追加

In [31]:
monthly_soldnum_T

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201905,201906,201907,201908,201909,201910,T2,T3,T6,T12
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,3.0,8.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,2.0,0.0,3.0,0.0,2.0,2.0,5.0,7.0,12.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,5.0,4.0,5.0,6.0,3.0,2.0,5.0,11.0,25.0,53.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,5.0,6.0,9.0,14.0,8.0,12.0,20.0,34.0,54.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,2.0,1.0,1.0,1.0,3.0,2.0,5.0,6.0,10.0,17.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
minmax_data = pd.merge(monthly_soldnum_T,store_sales,on='StoreId',how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201906,201907,201908,201909,201910,T2,T3,T6,T12,StoreSalesSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,0.0,2.0,0.0,1.0,0.0,1.0,1.0,3.0,8.0,107953.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46347.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,2.0,0.0,3.0,0.0,2.0,2.0,5.0,7.0,12.0,56431.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,4.0,5.0,6.0,3.0,2.0,5.0,11.0,25.0,53.0,87529.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,6.0,9.0,14.0,8.0,12.0,20.0,34.0,54.0,87.0,64902.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,1.0,1.0,1.0,3.0,2.0,5.0,6.0,10.0,17.0,125420.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51646.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,47944.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40016.0


In [33]:
minmax_data = pd.merge(minmax_data,ItemCategoryId_sales,on='ItemCategoryId',how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201907,201908,201909,201910,T2,T3,T6,T12,StoreSalesSum,ItemCategoryIdSoldSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,2.0,0.0,1.0,0.0,1.0,1.0,3.0,8.0,107953.0,293135.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46347.0,293135.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,3.0,0.0,2.0,2.0,5.0,7.0,12.0,56431.0,293135.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,5.0,6.0,3.0,2.0,5.0,11.0,25.0,53.0,87529.0,293135.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,9.0,14.0,8.0,12.0,20.0,34.0,54.0,87.0,64902.0,293135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,1.0,1.0,3.0,2.0,5.0,6.0,10.0,17.0,125420.0,2934.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51646.0,2934.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,47944.0,2934.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40016.0,2934.0


In [34]:
minmax_data = pd.merge(minmax_data,Category_sales,on='Category',how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201908,201909,201910,T2,T3,T6,T12,StoreSalesSum,ItemCategoryIdSoldSum,CategorySalesSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,0.0,1.0,0.0,1.0,1.0,3.0,8.0,107953.0,293135.0,427033.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46347.0,293135.0,427033.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,3.0,0.0,2.0,2.0,5.0,7.0,12.0,56431.0,293135.0,427033.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,6.0,3.0,2.0,5.0,11.0,25.0,53.0,87529.0,293135.0,427033.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,14.0,8.0,12.0,20.0,34.0,54.0,87.0,64902.0,293135.0,427033.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,1.0,3.0,2.0,5.0,6.0,10.0,17.0,125420.0,2934.0,27750.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51646.0,2934.0,27750.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,47944.0,2934.0,27750.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40016.0,2934.0,27750.0


In [35]:
minmax_data = pd.merge(minmax_data,Type_sales,on='Type',how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201909,201910,T2,T3,T6,T12,StoreSalesSum,ItemCategoryIdSoldSum,CategorySalesSum,TypeSalesSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,1.0,0.0,1.0,1.0,3.0,8.0,107953.0,293135.0,427033.0,293135.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,46347.0,293135.0,427033.0,293135.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,2.0,2.0,5.0,7.0,12.0,56431.0,293135.0,427033.0,293135.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,3.0,2.0,5.0,11.0,25.0,53.0,87529.0,293135.0,427033.0,293135.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,8.0,12.0,20.0,34.0,54.0,87.0,64902.0,293135.0,427033.0,293135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,3.0,2.0,5.0,6.0,10.0,17.0,125420.0,2934.0,27750.0,2934.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,51646.0,2934.0,27750.0,2934.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,47944.0,2934.0,27750.0,2934.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,40016.0,2934.0,27750.0,2934.0


In [36]:
minmax_data = pd.merge(
    minmax_data,StoreId_ItemCategoryId_sales,
    on=['StoreId','ItemCategoryId'],how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,201910,T2,T3,T6,T12,StoreSalesSum,ItemCategoryIdSoldSum,CategorySalesSum,TypeSalesSum,Store_ItemCat_SalesSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,0.0,1.0,1.0,3.0,8.0,107953.0,293135.0,427033.0,293135.0,27840.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,46347.0,293135.0,427033.0,293135.0,9543.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,2.0,2.0,5.0,7.0,12.0,56431.0,293135.0,427033.0,293135.0,12695.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,2.0,5.0,11.0,25.0,53.0,87529.0,293135.0,427033.0,293135.0,26495.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,12.0,20.0,34.0,54.0,87.0,64902.0,293135.0,427033.0,293135.0,15232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,2.0,5.0,6.0,10.0,17.0,125420.0,2934.0,27750.0,2934.0,550.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,51646.0,2934.0,27750.0,2934.0,30.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,47944.0,2934.0,27750.0,2934.0,89.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,40016.0,2934.0,27750.0,2934.0,89.0


In [37]:
minmax_data = pd.merge(
    minmax_data,StoreId_Category_sales,
    on=['StoreId','Category'],how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,T2,T3,T6,T12,StoreSalesSum,ItemCategoryIdSoldSum,CategorySalesSum,TypeSalesSum,Store_ItemCat_SalesSum,Store_Cat_SalesSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,1.0,1.0,3.0,8.0,107953.0,293135.0,427033.0,293135.0,27840.0,39998.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,46347.0,293135.0,427033.0,293135.0,9543.0,14836.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,2.0,5.0,7.0,12.0,56431.0,293135.0,427033.0,293135.0,12695.0,17706.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,5.0,11.0,25.0,53.0,87529.0,293135.0,427033.0,293135.0,26495.0,37160.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,20.0,34.0,54.0,87.0,64902.0,293135.0,427033.0,293135.0,15232.0,20238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,5.0,6.0,10.0,17.0,125420.0,2934.0,27750.0,2934.0,550.0,3380.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,51646.0,2934.0,27750.0,2934.0,30.0,229.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,47944.0,2934.0,27750.0,2934.0,89.0,512.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,40016.0,2934.0,27750.0,2934.0,89.0,710.0


In [38]:
minmax_data = pd.merge(
    minmax_data,StoreId_Type_sales,
    on=['StoreId','Type'],how='left')
minmax_data

,StoreId,ItemId,ItemCategoryId,Category,Type,201801,201802,201803,201804,201805,...,T3,T6,T12,StoreSalesSum,ItemCategoryIdSoldSum,CategorySalesSum,TypeSalesSum,Store_ItemCat_SalesSum,Store_Cat_SalesSum,StoreTypeSalesSum
0,0,1000001,100.0,5.0,1.0,6.0,3.0,1.0,2.0,0.0,...,1.0,3.0,8.0,107953.0,293135.0,427033.0,293135.0,27840.0,39998.0,27840.0
1,1,1000001,100.0,5.0,1.0,2.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,46347.0,293135.0,427033.0,293135.0,9543.0,14836.0,9543.0
2,2,1000001,100.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,...,5.0,7.0,12.0,56431.0,293135.0,427033.0,293135.0,12695.0,17706.0,12695.0
3,3,1000001,100.0,5.0,1.0,2.0,0.0,3.0,8.0,2.0,...,11.0,25.0,53.0,87529.0,293135.0,427033.0,293135.0,26495.0,37160.0,26495.0
4,4,1000001,100.0,5.0,1.0,0.0,2.0,1.0,2.0,2.0,...,34.0,54.0,87.0,64902.0,293135.0,427033.0,293135.0,15232.0,20238.0,15232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,13,3500001,350.0,6.0,6.0,1.0,2.0,1.0,0.0,3.0,...,6.0,10.0,17.0,125420.0,2934.0,27750.0,2934.0,550.0,3380.0,550.0
3056,14,3500001,350.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,51646.0,2934.0,27750.0,2934.0,30.0,229.0,30.0
3057,15,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,47944.0,2934.0,27750.0,2934.0,89.0,512.0,89.0
3058,16,3500001,350.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,40016.0,2934.0,27750.0,2934.0,89.0,710.0,89.0


In [40]:
minmax_data.to_csv('./csv/monthly_sales_T_MinMax.csv')